In [1]:
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics, new_dynamics2
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

In [2]:
length=[4, 4]
V = 1.0
h = 1.0
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)


ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))


hex_ = nk.machine.new_hex(np.array(length))

ma = nk.machine.RbmDimer(hi, hex_, alpha = 1, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)

yoyo
1


### set params to 0 (represent ground state for v=1)

In [3]:
ma._ws[:] = 0
ma._set_bare_parameters(
    ma._a, ma._b, ma._w, ma._as, ma._bs, ma._ws, ma._autom, ma._z2
)

### create Markov Chain 
number of markov chain is denoted by n_samples_

In [4]:
importlib.reload(nk)

<module 'netket' from '../netket/__init__.py'>

In [8]:
n_jobs = 6
length=[4, 4]
V = 1.0
h = 1.0

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op, ad2o_o, op_num, label_num  = f.dimer_flip1(length = np.array(length), return_info = True)
# op = f.dimer_flip_old(length = np.array(length))

ad2_bool = np.zeros([ad2o_o.shape[0], ad2o_o.shape[0]], dtype = np.bool)
for l in range(ad2o_o.shape[0]):
    label = ad2o_o[l]
    for op_ in label:
        ad2_bool[l,op_] = True
        
ma.hex.ad2o_o = ad2o_o.astype(np.int64)
ma.hex.ad2_bool = ad2_bool
hex_ = nk.machine.new_hex(np.array(length))




sweep_size = 1000
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op, length = length, n_chains=1, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sa_mul.reset()
_ = sa_mul.generate_samples(1000) # discard the begginings of metropolis sampling.

number of core : 6
transtion2
transtion2
transtion2
transtion2
transtion2
transtion2


In [6]:
n_samples_ = 10000
samples_state = sa_mul.generate_samples(int(n_samples_ / 1))
samples_state = samples_state.reshape(-1, ma.hilbert.size)
print('get samples')

get samples


In [9]:
samples2 = np.load("P_new.npy")

In [10]:
import scripts.dynamics as Dynamics
from importlib import reload
reload(Dynamics)
t_list = np.arange(0,20,0.1)

# d = Dynamics.new_dynamics(op, ma)
# %time P_ = d.multiprocess(samples_state, 300, 12) 
# d = Dynamics.new_dynamics3(op, ma)
# %time P_ = d.multiprocess(samples_state, t_list, 12) 
import scripts.dynamics_4 as sdy
reload(sdy)


d = sdy.new_dynamics_one(op, ma)
P_list = d.multiprocess(samples2[:10**4], t_list,6) 

In [11]:
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))

In [12]:
from importlib import reload
reload(f)


operators = f.return_vison(hi, np.arange(hex_.lattice_coor.shape[0]))

In [13]:
hex_.l[0]

4

In [14]:
v1 = np.array([0.5, -np.tan(np.pi / 6) * 1/2])
v2 = hex_.lattice_coor[2] - hex_.lattice_coor[1]

# v1 = np.round(v1,5)

In [15]:
x = np.arange(3*length[0])
y = np.arange(length[1])

In [16]:
A = np.array([v1,v2])
[vb1, vb2] = np.linalg.inv(A.T)

In [27]:
x = np.arange(3*length[0])
y = np.arange(length[1])

xx, yy = np.meshgrid(x,y)
xx = xx.reshape(-1)
yy = yy.reshape(-1)

edge_coor_array = (hex_.ProcessPeriodic(xx[:,None] * v1[None,:] + yy[:,None]  * v2[None,:] + hex_.lattice_coor[0]))

In [28]:
corrs = hex_.coor_to_lattice_num(edge_coor_array)
corrs_ = corrs.reshape(y.shape[0], x.shape[0])
operators = f.return_vison(hi, corrs)

In [92]:
reload(f)
vison_corr, vison_std, num_samples = f.cal_vison_corr(operators, P_list, hex_, t_list, False)

0
1
2
3


In [93]:
vison_prime = f.process_symm_vison(vison_corr, corrs, hex_)

In [89]:
reload(f)
vison_momentum = f.vison_fourier_simple(vison_prime, hex_, edge_coor_array)

In [79]:
L1, L2 = 12,4
for i1 in range(L1):
    for i2 in range(L2):
        for l1 in range(L1):
            for l2 in range(L2):
                print(vison_prime[i1,i2,l1,l2][:5])
                print(vison_corr[i1,i2,l1,l2][:5])    
                print(i1,i2,l1,l2)
                print("\n")

[1.        0.9422125 0.8947375 0.854725  0.82045  ]
[1.     0.9442 0.899  0.86   0.8258]
0 0 0 0


[0.3325    0.3318125 0.3302375 0.326625  0.3237125]
[0.3442 0.3458 0.3404 0.3384 0.3316]
0 0 0 1


[0.194375  0.1950625 0.1967875 0.19645   0.195675 ]
[0.1974 0.1998 0.1972 0.1962 0.1982]
0 0 0 2


[0.3325    0.3318125 0.3302375 0.326625  0.3237125]
[0.3344 0.3332 0.3328 0.3276 0.3218]
0 0 0 3


[0.6663    0.6462875 0.629025  0.6123375 0.5979125]
[0.6752 0.6516 0.6372 0.6172 0.6034]
0 0 1 0


[0.210025  0.2097875 0.2097125 0.2095125 0.2078375]
[0.214  0.2208 0.2194 0.2196 0.2178]
0 0 1 1


[0.2063    0.2066375 0.2079125 0.2065    0.206075 ]
[0.211  0.2048 0.2052 0.205  0.2044]
0 0 1 2


[0.6662    0.6461375 0.628475  0.6113875 0.5962375]
[0.675  0.6564 0.6362 0.6168 0.6018]
0 0 1 3


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
0 0 2 0


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
0 0 2 1


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
0 0 2 2


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
0 0 2 3


[0.003325  0.0034375 0.0028875 0

[0.0108 0.0096 0.0072 0.0108 0.0082]
3 0 1 0


[0.6675    0.6468375 0.6274875 0.611025  0.5954125]
[0.6696 0.6408 0.621  0.6084 0.5922]
3 0 1 1


[-1.08420217e-19 -2.50000000e-05 -3.75000000e-05  1.87500000e-04
 -1.25000000e-05]
[0.0106 0.0076 0.0084 0.0086 0.0076]
3 0 1 2


[0.070525  0.0717    0.0724375 0.0725625 0.071825 ]
[0.0718 0.0696 0.0718 0.0732 0.0766]
3 0 1 3


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
3 0 2 0


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
3 0 2 1


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
3 0 2 2


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
3 0 2 3


[1.        0.9422125 0.8947375 0.854725  0.82045  ]
[1.     0.9412 0.8954 0.8552 0.8184]
3 0 3 0


[0.3325    0.3318125 0.3302375 0.326625  0.3237125]
[0.3242 0.3272 0.3296 0.3236 0.317 ]
3 0 3 1


[0.194375  0.1950625 0.1967875 0.19645   0.195675 ]
[0.1988 0.199  0.2002 0.1928 0.1916]
3 0 3 2


[0.3325    0.3318125 0.3302375 0.326625  0.3237125]
[0.329  0.3272 0.326  0.3224 0.3172]
3 0 3 3


[0.6663    0.6462875 0.629025  0.6123375 0.5979125]
[

[-0.0798 -0.0802 -0.081  -0.0794 -0.0812]
6 0 1 0


[0.0697    0.068925  0.0686    0.0702625 0.0699875]
[0.0586 0.0606 0.0588 0.0622 0.0568]
6 0 1 1


[0.210325 0.2095   0.20895  0.208825 0.20745 ]
[0.2008 0.2038 0.199  0.2016 0.201 ]
6 0 1 2


[-0.2071    -0.207975  -0.2069375 -0.2049875 -0.2035125]
[-0.2184 -0.2182 -0.212  -0.2058 -0.2024]
6 0 1 3


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
6 0 2 0


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
6 0 2 1


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
6 0 2 2


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
6 0 2 3


[-0.003325  -0.0034375 -0.0028875 -0.003     -0.0019375]
[-0.0066 -0.0082 -0.0108 -0.0078 -0.009 ]
6 0 3 0


[-0.3338    -0.3323125 -0.3302    -0.327225  -0.3238   ]
[-0.3244 -0.3242 -0.325  -0.3238 -0.3212]
6 0 3 1


[-0.3337    -0.33295   -0.329875  -0.3262875 -0.32155  ]
[-0.3278 -0.3292 -0.3308 -0.329  -0.3238]
6 0 3 2


[ 0.000625  -0.0002875 -0.0015    -0.0013625 -0.0015625]
[0.0048 0.0074 0.0086 0.0042 0.0054]
6 0 3 3


[0.000e+00 1.125e-04 1.500e-04 2.500e-0

[0.002425  0.002075  0.0015    0.0025125 0.0018   ]
[-0.005  -0.004  -0.0024 -0.003  -0.0022]
9 1 3 2


[0.195625  0.1950125 0.1954875 0.196525  0.195975 ]
[0.188  0.1846 0.1858 0.1866 0.1914]
9 1 3 3


[-0.2071    -0.207975  -0.2069375 -0.2049875 -0.2035125]
[-0.2182 -0.2142 -0.2144 -0.2136 -0.211 ]
9 1 4 0


[-0.068175  -0.068325  -0.0678625 -0.067475  -0.0680625]
[-0.0788 -0.077  -0.0786 -0.0714 -0.0724]
9 1 4 1


[0.0697    0.068925  0.0686    0.0702625 0.0699875]
[0.0602 0.0606 0.064  0.0634 0.067 ]
9 1 4 2


[0.210325 0.2095   0.20895  0.208825 0.20745 ]
[0.2038 0.2032 0.2044 0.206  0.2034]
9 1 4 3


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
9 1 5 0


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
9 1 5 1


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
9 1 5 2


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
9 1 5 3


[ 0.000625  -0.0002875 -0.0015    -0.0013625 -0.0015625]
[ 0.0056  0.0016 -0.0004 -0.0038 -0.0066]
9 1 6 0


[-0.003325  -0.0034375 -0.0028875 -0.003     -0.0019375]
[-0.0036 -0.003  -0.001  -0.0018  0.0002]
9 



[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 5 3


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 6 0


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 6 1


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 6 2


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 6 3


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 7 0


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 7 1


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 7 2


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 7 3


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 8 0


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 8 1


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 8 2


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 8 3


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 9 0


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 9 1


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 9 2


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 9 3


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 10 0


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 10 1


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 10 2


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 10 3


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
11 3 11 0


[0.

In [236]:
n1, n2 = 0,1
print(corrs[n1 + 12 * n2] + 1)

n1, n2 = 0,3
print(corrs[n1 + 12 * n2]+ 1)

n1, n2 = 1,2
print(corrs[n1 + 12 * n2]+ 1)

8
11
12


In [70]:
# m1, m2, n1, n2 = (4, 2, 3, 0)
# m1_, m2_, n1_, n2_ = m1, m2, n1, n2
# for i in range(12):
#     m1__, m2__, n1__, n2__ = m1_, m2_, n1_, n2_
#     for j in range(4):
#         print(m1__, m2__, n1__, n2__)
#         print(vison_corr[m1__, m2__, n1__, n2__][:5])
#         m1__, m2__, n1__, n2__ = (m1__, m2__+1, n1__, n2__+1)
#         m1__, m2__ = m1__%12, m2__%4
#         n1__, n2__ = n1__%12, n2__%4
        
#     m1_, m2_, n1_, n2_ = (m1_+1, m2_, n1_+1, n2_)
#     m1_, m2_ = m1_%12, m2_%4
#     n1_, n2_ = n1_%12, n2_%4

In [63]:
m1, m2, n1, n2 = (0, 0, 1, 0)
m1_, m2_, n1_, n2_ = m1, m2, n1, n2
for i in range(4):
    m1__, m2__, n1__, n2__ = m1_, m2_, n1_, n2_
    for j in range(4):
        print(m1__, m2__, n1__, n2__)
        print(vison_corr[m1__, m2__, n1__, n2__][:5])
        m1__, m2__, n1__, n2__ = (m1__, m2__+1, n1__, n2__+1)
        m1__, m2__ = m1__%12, m2__%4
        n1__, n2__ = n1__%12, n2__%4
        
    m1_, m2_, n1_, n2_ = (m1_+3, m2_-1, n1_+3, n2_-1)
    m1_, m2_ = m1_%12, m2_%4
    n1_, n2_ = n1_%12, n2_%4
    

0 0 1 0
[0.6752 0.6516 0.6372 0.6172 0.6034]
0 1 1 1
[0.6722 0.6534 0.6324 0.6182 0.6052]
0 2 1 2
[0.6732 0.6514 0.6366 0.6212 0.6086]
0 3 1 3
[0.6594 0.6412 0.625  0.6064 0.595 ]
3 3 4 3
[0.6694 0.6548 0.6388 0.6264 0.6062]
3 0 4 0
[0.6524 0.6356 0.6218 0.6042 0.5884]
3 1 4 1
[0.6668 0.6482 0.6294 0.6086 0.5908]
3 2 4 2
[0.6644 0.648  0.6254 0.612  0.5928]
6 2 7 2
[0.6562 0.637  0.619  0.6046 0.5894]
6 3 7 3
[0.6538 0.63   0.6158 0.6    0.5928]
6 0 7 0
[0.672  0.6536 0.635  0.6176 0.6014]
6 1 7 1
[0.6702 0.6504 0.6294 0.6196 0.6078]
9 1 10 1
[0.673  0.645  0.625  0.6114 0.5986]
9 2 10 2
[0.6626 0.6398 0.6256 0.6044 0.594 ]
9 3 10 3
[0.6656 0.6472 0.632  0.6114 0.6004]
9 0 10 0
[0.6744 0.6534 0.6342 0.6116 0.5912]


In [109]:
process_index(1,-1)

(9, 3)

In [112]:
def process_index(i1, i2, L = 4):
    if i2 == -1:
        i1 = (i1 - L) 
        i2 = L-1
    elif i2 == 4:
        i1 = (i1 + L) 
        i2 = 0
    return i1% (L*3) , i2 % L
    

In [11]:
num_samples = (P_list[:,:,0]!=0).sum(axis=1)
P_list_ = P_list.reshape(-1,P_list.shape[-1])
sections1 = np.arange(P_list.shape[1])
sections2 = np.zeros(P_list_.shape[0])

_, mels1, _ = operators[0].get_conn_flattened(P_list[0,:,:], sections1)
sub1 = mels1.mean().real
mels1 = mels1.real

size = len(operators)

dimer_corr = np.zeros((size,t_list.shape[0]))
dimer_std = np.zeros((size,t_list.shape[0]))


for s in range(1):
    print(s)
    
    if operators[s]:
        sub2 = operators[s].get_conn_flattened(P_list[0,:,:], sections1)[1].mean().real
        _, mels2_, _ = operators[s].get_conn_flattened(P_list_, sections2)
        mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real
        dimer_corr[s] = (mels2 * mels1).mean(axis=1)
        dimer_std[s] = (mels2 * mels1).std(axis=1)
    else:
        dimer_corr[s] = 0
        dimer_std[s] = 0

yo
0
yo
yo


In [29]:
s = 0
for i in range(P_list.shape[-1]):
    s += np.mean(P_list[:,:,i] * P_list[0,:,i],axis=-1)   
s/=P_list.shape[-1]

In [39]:
name = 'h={}V={}l={}'.format(h, V, length)

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
s = []
for j in range(100):

    P_list = np.load(parentdir+f'/save/dynamics/{name}/P_n=1.0e+04_{j}.npy')
    # new_P_list = np.zeros((P_list.shape[0], P_list.shape[1] * T.shape[0] * T.shape[1], P_list.shape[-1]))

    # for t in range(len(new_P_list)):
    #     new_P_list[t] = f.permute(P_list[t], T, color)
    s.append(0)
    for i in range(P_list.shape[-1]):
        s[j] += np.mean(P_list[:,:,i] * P_list[0,:,i],axis=-1)   
    s[j]/=P_list.shape[-1]
    

FileNotFoundError: [Errno 2] No such file or directory: '/home/keisuke/Documents/dimermaster/save/dynamics/h=1.0V=1.0l=[4, 4]/P_n=1.0e+04_30.npy'

In [ ]:
dimer_std[0] / np.sqrt(10**5)

In [ ]:
from plotly import graph_objects as go

In [ ]:

data_list = []

for correlation in dimer_corr:
    data_list.append(go.Scatter(
        x = t_list,
        y = correlation * np.sign(correlation[0])
    ))
    

Fig = go.Figure(data = data_list)

In [ ]:
Fig.show()

In [26]:
file_name = 'vison{}_{:.0e}_T={:.1f}_2.dat'.format(length,n_samples_, t_list[-1])
f.save_corr(file_name, dimer_corr, t_list)

In [19]:
tau_list = []
N = 2000
corr1 = []
corr2 = np.zeros((81,N),dtype=np.complex128)
n = -1
import re
with open('K4_4V1.0st.dat') as fe:
    for line in fe:
        
        if re.findall(r'#\d', line):
            i = 0
            n += 1
        else:
            try:
                match = re.findall(r'[\d,\.]+', line)
                tau_list.append(float(match[1]))
                corr1.append(float(match[2]))
                corr2[n,i] = float(match[2]) + 1j*float(match[3])
                i+=1
            except:
                pass